In [1]:
from scipy.misc import *
import numpy as np
import pylab as plb
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.integrate import quad
from scipy.integrate import nquad
from scipy.misc import derivative
from ipywidgets import interact, interactive, fixed
from ipywidgets import widgets
from matplotlib.widgets import Slider, Button, RadioButtons, CheckButtons, TextBox # Matplotlib widgets
import pandas as pd
import emcee
import corner
from astropy import table as Table # For fast and easy reading / writing with tables using numpy library
from galpy.potential import MiyamotoNagaiPotential, NFWPotential, RazorThinExponentialDiskPotential, BurkertPotential # GALPY potentials

In [2]:
# In[ ]:

tt=Table.Table.read('coordinates.txt', format='ascii.tab') # importando los datos de las imágenes

#Importando coordenadas de las imágenes deflectadas
theta1=tt['theta1'] 
theta2=tt['theta2']
sigma=tt['sigma']

theta=np.zeros(len(theta1),float)
for i in range(len(theta1)):
    theta[i]=np.sqrt(theta1[i]**2+theta2[i]**2)

tt=Table.Table.read('Cosmological_distances.txt', format='ascii.tab') # importando los datos de distancias cosmológicas

#Importando distancias cosmológicas y Sigma Crítico
D_ds=tt['D_ds'][0] 
D_d=tt['D_d'][0]
D_s=tt['D_s'][0]
SIGMA_CRIT=tt['SIGMA_CRIT'][0]


FC = np.pi/(180*3600) #conversion factor between arcs and radians


In [3]:
def THETA(H,K,SIGMA_0, H_R):
    GRADPOT1disk_exp = np.zeros((len(theta1)), float)
    GRADPOT2disk_exp = np.zeros((len(theta1)), float)
    GRADPOT1 = np.zeros((len(theta1)), float)
    GRADPOT2 = np.zeros((len(theta1)), float)
    THETA1 = np.zeros((len(theta1)), float)
    THETA2 = np.zeros((len(theta1)), float)
    THETA_teor = np.zeros((len(theta1)), float)
    h=H*FC
    k=K*FC
    Beta1 = h
    Beta2 = k
    def POTDEFdisk_exp1(x):
        def integ(TheTa, theta):
            Sigma = SIGMA_0*np.exp(-D_d*TheTa/H_R) #Volumetric density
            return 2*TheTa*np.log(THETA/TheTa)*Sigma/(SIGMA_CRIT**3)
        THETA = np.sqrt(x**2 + theta2[l]**2)
        x = quad(integ, 0, theta[l], limit=100, args=(theta))[0]
        return x
    def POTDEFdisk_exp2(x):
        def integ(TheTa, theta):
            Sigma = SIGMA_0*np.exp(-D_d*TheTa/H_R) #Volumetric density
            return 2*TheTa*np.log(THETA/TheTa)*Sigma/(SIGMA_CRIT**3)
        THETA = np.sqrt(x**2 + theta1[l]**2)
        x = quad(integ, 0, theta[l], limit=100, args=(theta))[0]
        return x
    for l in range(len(theta1)):
        GRADPOT1disk_exp[l]= derivative(POTDEFdisk_exp1, theta1[l], dx=1e-9, order=7)
        GRADPOT2disk_exp[l]= derivative(POTDEFdisk_exp2, theta2[l], dx=1e-9, order=7)
    for l in range(len(theta1)):
        GRADPOT1[l]=(SIGMA_CRIT**2)*(GRADPOT1disk_exp[l])
        GRADPOT2[l]=(SIGMA_CRIT**2)*(GRADPOT2disk_exp[l])
    for l in range(len(theta1)):
        THETA1[l] = Beta1+GRADPOT1[l]
        THETA2[l] = Beta2+GRADPOT2[l]
    #Graphics of source and images
    fig = plt.figure()
#    ax = fig.add_subplot(111)
    plt.rcParams['figure.figsize'] =(5,5)
    plb.plot(Beta1/FC, Beta2/FC, 'or')
    plb.plot(theta1/FC, theta2/FC, 'ob')
    plb.plot(THETA1/FC, THETA2/FC, 'og')
    plb.xlim(-2.5,2.5)
    plb.ylim(-2.5,2.5)
    plb.legend(['Source', 'Observational data', 'model values'], loc='upper right', fontsize=15)
#    ax.set_aspect('equal', adjustable='box')

In [4]:
interact(THETA, H=widgets.FloatSlider(min=-0.2, max=0.2, step=0.01, value=-0.15),
        K=widgets.FloatSlider(min=-0.2, max=0.2, step=0.01, value=-0.15),
        H_R=widgets.FloatSlider(min=2, max=50, step=0.2, value=3),
        SIGMA_0 = widgets.FloatSlider(min=1e8, max=120e8, step=0.5e8, value=1e8));

interactive(children=(FloatSlider(value=-0.15, description='H', max=0.2, min=-0.2, step=0.01), FloatSlider(val…